In [6]:
from __future__ import print_function
import cv2 
import numpy as np
import argparse
from math import sqrt

In [7]:
img1 = cv2.imread('eiffel_1.jpg')  
img2 = cv2.imread('eiffel_3.jpg') 

### Detect keypoints and compute descriptors using AKAZE

In [9]:
akaze = cv.AKAZE_create()
kpts1, desc1 = akaze.detectAndCompute(img1, None)
kpts2, desc2 = akaze.detectAndCompute(img2, None)

### Use brute-force matcher to find 2-nn matches

In [10]:
matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_BRUTEFORCE_HAMMING)
nn_matches = matcher.knnMatch(desc1, desc2, 2)

### Use 2-nn matches and ratio criterion to find correct keypoint matches

In [11]:
matched1 = []
matched2 = []
nn_match_ratio = 0.8 # Nearest neighbor matching ratio
for m, n in nn_matches:
    if m.distance < nn_match_ratio * n.distance:
        matched1.append(kpts1[m.queryIdx])
        matched2.append(kpts2[m.trainIdx])

### Check if our matches fit in the homography model


In [17]:
fs = cv2.FileStorage(cv2.samples.findFile(args.homography), cv2.FILE_STORAGE_READ)
homography = fs.getFirstTopLevelNode().mat()

usage: ipykernel_launcher.py [-h] [--input1 INPUT1] [--input2 INPUT2]
                             [--homography HOMOGRAPHY]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/eduard/.local/share/jupyter/runtime/kernel-eaefab8a-cee3-4873-8099-6f54acb7bea3.json


SystemExit: 2

In [14]:
inliers1 = []
inliers2 = []
good_matches = []
inlier_threshold = 2.5 # Distance threshold to identify inliers with homography check
for i, m in enumerate(matched1):
    col = np.ones((3,1), dtype=np.float64)
    col[0:2,0] = m.pt
    col = np.dot(homography, col)
    col /= col[2,0]
    dist = sqrt(pow(col[0,0] - matched2[i].pt[0], 2) +\
                pow(col[1,0] - matched2[i].pt[1], 2))
    if dist < inlier_threshold:
        good_matches.append(cv.DMatch(len(inliers1), len(inliers2), 0))
        inliers1.append(matched1[i])
        inliers2.append(matched2[i])

NameError: name 'homography' is not defined